In [63]:
from time import sleep
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common import keys
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import en_core_web_sm
import fr_core_news_sm
# Import summarize from gensim
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords# Import the library
# to convert MSword doc to txt for processing.
import docx2txt
nlp_job = fr_core_news_sm.load()
nlp_resume=fr_core_news_sm.load()
from dotenv import load_dotenv,dotenv_values
from getpass import getpass

load_dotenv()

config = dotenv_values(".env")



def get_jobs_links(job_query,user,pwd):
    """[summary]

    Args:
        job_query ([type]): [description]
    """
    browser=webdriver.Chrome("chromedriver.exe")
    browser.get("https://www.linkedin.com")

    username=browser.find_element_by_id("session_key")
    username.send_keys(user)
    password=browser.find_element_by_id("session_password")
    password.send_keys(pwd)

    login_button=browser.find_element_by_class_name("sign-in-form__submit-button")
    login_button.click()

    browser.get(job_query)
    sleep(3)
    jobs=browser.find_elements_by_class_name("job-card-container")
    
    
    
    jobs_links=[]
    jobs_head=[]
    for i in jobs:
        jobs_links.append(i.find_elements_by_tag_name('a')[0].get_attribute('href'))
        jobs_head.append(i.find_element_by_class_name("artdeco-entity-lockup__content").text)
    return browser,jobs_links,jobs_head


def scrape_job(browser,job_link):
    """
    """
    browser.get(job_link)
    sleep(3)
    more_button=browser.find_element_by_class_name("artdeco-card__action")
    more_button.click()
    sleep(3)
    content=browser.find_elements_by_class_name('jobs-box__html-content')[0].text
    # print(content)
    return content

def get_keywords_from_job(job_description):
    """[summary]

    Args:
        job_description ([type]): [description]
    """
    keyword = []
    stopwords = list(STOP_WORDS)
    pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
    for token in job_description:
        if(token.text in stopwords or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            keyword.append(token.text)
    return keyword
            
def get_n_common_words(keywords,n):
    freq_word = Counter(keywords)
    print(freq_word.most_common(n))    
    return freq_word   

def get_sent_strength(freq_word):
    sent_strength={}
    for sent in doc.sents:
        for word in sent:
            if word.text in freq_word.keys():
                if sent in sent_strength.keys():
                    sent_strength[sent]+=freq_word[word.text]
                else:
                    sent_strength[sent]=freq_word[word.text]
    print(sent_strength)
    return sent_strength
def summurize_sent(sent_strength,ratio=3):
    summarized_sentences = nlargest(ratio, sent_strength, key=sent_strength.get)
    # print(summarized_sentences)
    final_sentences = [ w.text for w in summarized_sentences ]
    summary = ' '.join(final_sentences)
    # print(summary)
    return summary


def match_resume_and_job(job_description, resume,head,i):
    text_list = [job_description, resume]
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(text_list)
    from sklearn.metrics.pairwise import cosine_similarity
    # get the match percentage
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 2) # round to two decimal
    # print(keywords(job_description, ratio=0.25)) 
    # gives you the keywords of the job description
    return head, matchPercentage
from urllib.parse import quote
 

def matching_pipeline(job_link,user,pwd,resume_path):
    resume_content = docx2txt.process("CV_Brice_FOTZO.docx")
    browser,links,heads=get_jobs_links(job_link,user,pwd)
    sleep(3)
    cpt=1
    features=[]
    for head,job in zip(heads,links):
        job_content=scrape_job(browser,job)
        score=match_resume_and_job(job_content,resume_content,head,cpt)[1]
        cpt=cpt+1
        features.append({"head":head,"score":score})


    features_sorted=sorted(features, key=lambda kv: kv['score'], reverse=True)
    # print(features_sorted)
    for i,job in enumerate(features_sorted):
        print("Job {} - Score {} %".format(i,str(job['score'])))
        print("Infos",job['head'])
        print("----------------------------------------------------------------------------")
if __name__=="__main__":
    userN=input("Email/Nom d'utilisateur : ")
    pswd=getpass("Mot de passe : ")
    job_query_online = input("Enter Job description : ")
    keyword_job=quote(job_query_online)
    jobs="https://www.linkedin.com/jobs/search/?keywords="+keyword_job+"&location=France"
    matching_pipeline(job_link=jobs,user=userN,pwd=pswd,resume_path="CV_Brice_FOTZO.docx")

Job 0 - Score 57.9 %
Infos Développeur web Odoo F/H
2CRSi
Strasbourg, Grand Est, France
----------------------------------------------------------------------------
Job 1 - Score 54.25 %
Infos Chef de Projet Technique WEB
JEMS
Neuilly-sur-Seine, Île-de-France, France
----------------------------------------------------------------------------
Job 2 - Score 50.6 %
Infos Développeur Web Java
NORGAY
Brest, Bretagne, France
----------------------------------------------------------------------------
Job 3 - Score 50.51 %
Infos Développeur Web Back End H/F
Capgemini
Mougins, Provence-Alpes-Côte d’Azur, France
----------------------------------------------------------------------------
Job 4 - Score 45.87 %
Infos Developpeur Web
ALD Automotive
Clichy, Île-de-France, France
----------------------------------------------------------------------------
Job 5 - Score 43.68 %
Infos Développeur web (CDI)
GARNEM SAS
Avignon, Provence-Alpes-Côte d’Azur, France
----------------------------------------

In [56]:
dico=[{'head': 'Data Scientist\nLa Banque Postale\nParis, Île-de-France, France', 'score': 61.43}, {'head': 'Pricing Data Scientist\nDassault Systèmes\nVélizy-Villacoublay, Île-de-France, France', 'score': 60.65}, {'head': 'Data Scientist\nGroupe Caisse des Dépôts\nParis, Île-de-France, France', 'score': 60.2}, {'head': 'Data Scientist\nSocio Data Management\nParis et périphérie', 'score': 57.78}, {'head': 'Data Scientist\nNTICO Technology\nLille, Hauts-de-France, France', 'score': 51.02}, {'head': "Data Analyst / Data Scientist (H/F)\nCSA (Institut d'Etudes)\nPuteaux, Île-de-France, France", 'score': 50.21}, {'head': 'Data Scientist-(H/F)\nSociété Générale\nFontenay-sous-Bois, Île-de-France, France', 'score': 48.5}]

In [60]:
for i,job in enumerate(dico):
    print(job['score'])

61.43
60.65
60.2
57.78
51.02
50.21
48.5


In [64]:
job_description="""
Entité de rattachement : Banque des Territoires - Direction des Prêts

 

Description de l’entité

La Caisse des Dépôts et ses filiales forment un groupe public au service de l'intérêt général et du développement économique des territoires. Elle agit en appui des politiques publiques conduites par l'Etat et les collectivités locales, prioritairement pour répondre aux grands défis de société : transition territoriale, transition énergétique et écologique, transition démographique et transition numérique. Elle assure également la gestion de grands mandats publics (fonds privés, retraite, financement du logement social…) et intervient comme banquier du service public de la justice et de la sécurité sociale.

Elle est, au travers de la Direction des prêts de sa Banque des territoires, le financeur principal du secteur du logement social avec près de 80% de sa dette représentant 160Mds€ d’encours.

 

Le service :

Au sein de la direction des prêts, le service Pilotage et Statistiques a pour missions de valoriser les données nécessaires au pilotage de l’activité de prêt, pour comprendre les dynamiques d’activité et y participer par des approches innovantes (apporter de la performance, mieux comprendre la concurrence, mieux cibler les prospects etc). Il est à ce titre Product Owner de Diapason, qui est le SI permettant d’avoir une vision 360 de l’activité globale de prêt (le caractère industriel de l’activité de prêt conduit par nature à disposer de très importants volumes de données d’intérêt).

Il est par ailleurs chargé de réaliser et mettre en œuvre :

Un ensemble de reportings réguliers alimentant le pilotage stratégique de l’activité ;
Les prévisions d’activité de prêt (signatures – versements), alimentant notamment la fixation des objectifs des DR, les PMT (BDT – Fonds d’Epargne (FE) et Etablissement Public CDC) ou encore les travaux relatifs à l’équilibre ALM des Fonds d’Epargne ;
Le développement d’approches quantitatives innovantes (économétrie – machine learning – deep learning) afin d’éclairer ou faciliter l’activité opérationnelle de prêt. C’est notamment sur ce dernier champ que s’inscrit cette offre de poste, sans être exclusive.
 

L’équipe :

Au sein d’une équipe de 5 personnes, vous serez amené(e) à travailler au service « Pilotage et Statistiques ».

 

Le mot du Manager :

« Ce département est passionnant : pouvoir déployer les méthodes quantitatives innovantes, sur des données présentant des volumes importants compte-tenu du caractère industriel de l’activité prêteur, approches appliquées à des problématiques concrètes et opérationnelles en lien avec le caractère stratégique de cette activité et sa prise directe avec l’intérêt général, c’est ce que permet vraiment le poste proposé !

Vous pourrez apporter une vision data et machine learning, en réalisant des études, en implémentant des services innovants. Direction tremplin de la CDC, ce poste vous permettra de développer vos connaissances en data science, vous offrant ainsi de réelles perspectives d’évolution en interne ».

 

Description du poste

 

Positionnement :

Intégré au sein de la Direction des Prêts, vous serez en charge des missions principales suivantes :

 

La mise en œuvre d'algorithmes de textmining sur les documents à disposition de la Direction des Prêts (pièce justificative des clients, etc.), l’extraction automatique d'informations, la classification de documents, la correction d'erreurs
L’analyse de données / modèles prédictifs (statistiques descriptives, la classification, l’analyse, la construction de modèles prédictifs)
Divers travaux de collecte / croisement de base de données
 

Missions secondaires :

 

Mise en œuvre d'outils de webscrapping (développer le champ des informations récupérées de manière automatique sur le net)
Statistique spatiale / cartographie / exploitation de base de données géographiques
 

Profil du candidat

 

Niveau de formation :

Bac+5 Ecole d’ingénieur, de formation scientifique

 

Expérience professionnelle :

Minimum 3 ans d’expérience dans la programmation et les mathématiques appliquées

 

Compétences :

Une expérience avérée en modélisation / programmation / mathématiques appliquées
Une bonne connaissance de la programmation (python)
Une bonne maîtrise des statistiques classiques et des dernières évolutions dans le domaine de l'IA et du deep learning
De bonnes capacités de mise en œuvre des outils d'IA dans le domaine du traitement du texte, de l'image et de données plus classiques.
Une bonne connaissance de différentes approches statistiques, dont l'utilisation de réseaux de neurones, est nécessaire
 

Une maîtrise des librairies (tensorflow / scikitlearn / pandas / numpy) et du machine learning / deep learning serait un plus

 

Qualités :

Vous êtes également curieux, rigoureux, autonome, force de propositions innovantes, apte à construire des solutions opérationnelles / industrialisables pertinentes? Rejoignez-nous!

 

Localisation du poste : Paris 13


Merci d'adresser votre candidature (CV + lettre de motivation) en français

 

Le recrutement à la CDC est fondé sur les compétences, sans distinction d’origine, d’âge, ni de genre. Tous nos postes sont ouverts aux personnes en situation de handicap.

Niveau hiérarchique
Confirmé

Secteur
Administration publique
Type d’emploi
Temps plein

Fonctions
Ingénierie  Technologies de l’information"""

In [65]:
def get_keywords_from_job(job_description):
    """[summary]

    Args:
        job_description ([type]): [description]
    """
    keyword = []
    stopwords = list(STOP_WORDS)
    pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
    for token in job_description:
        if(token.text in stopwords or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            keyword.append(token.text)
    return keyword
            
def get_n_common_words(keywords,n):
    freq_word = Counter(keywords)
    print(freq_word.most_common(n))    
    return freq_word   

def get_sent_strength(freq_word):
    sent_strength={}
    for sent in doc.sents:
        for word in sent:
            if word.text in freq_word.keys():
                if sent in sent_strength.keys():
                    sent_strength[sent]+=freq_word[word.text]
                else:
                    sent_strength[sent]=freq_word[word.text]
    print(sent_strength)
    return sent_strength
def summurize_sent(sent_strength,ratio=3):
    summarized_sentences = nlargest(ratio, sent_strength, key=sent_strength.get)
    # print(summarized_sentences)
    final_sentences = [ w.text for w in summarized_sentences ]
    summary = ' '.join(final_sentences)
    # print(summary)
    return summary

In [68]:
nlp_resume=fr_core_news_sm.load()
# doc=nlp_resume(str(resume))
get_keywords_from_job(nlp_resume(job_description))

['Entité',
 'rattachement',
 'Banque',
 'Territoires',
 'Direction',
 'Prêts',
 'Description',
 'l’',
 'entité',
 'Caisse',
 'Dépôts',
 'filiales',
 'forment',
 'groupe',
 'public',
 'service',
 'intérêt',
 'général',
 'développement',
 'économique',
 'territoires',
 'agit',
 'appui',
 'politiques',
 'publiques',
 'conduites',
 'Etat',
 'collectivités',
 'locales',
 'répondre',
 'grands',
 'défis',
 'société',
 'transition',
 'territoriale',
 'transition',
 'énergétique',
 'écologique',
 'transition',
 'démographique',
 'transition',
 'numérique',
 'assure',
 'gestion',
 'grands',
 'mandats',
 'publics',
 'fonds',
 'privés',
 'retraite',
 'financement',
 'logement',
 'social',
 '…',
 'intervient',
 'banquier',
 'service',
 'public',
 'justice',
 'sécurité',
 'sociale',
 'est',
 'travers',
 'Direction',
 'prêts',
 'Banque',
 'territoires',
 'financeur',
 'principal',
 'secteur',
 'logement',
 'social',
 'dette',
 'représentant',
 '160Mds€',
 'encours',
 'service',
 'sein',
 'direction',